# Publish Tweets to Kafka Topic

Add in your Twitter credentials below

In [ ]:
import tweepy
from tweepy import OAuthHandler

consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

from kafka import KafkaConsumer, KafkaProducer
import json

producer = KafkaProducer(bootstrap_servers='localhost:9092')

Send a test message to the Kafka topic "twitterstream"

In [ ]:
producer.send ('twitterstream',json.dumps('test message').encode('utf-8'))

Start streaming Twitter feeds. Adjust hashtags as needed in the following line:
<pre>twitter_stream.filter(track=['#bigdata'])</pre>

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('python.json', 'a') as f:
                f.write(data)
                producer.send ('twitterstream',json.dumps(data).encode('utf-8'))
                return True
        except BaseException as e:
            #print(&quot;Error on_data: %s&quot; % str(e))
            return True
 
    def on_error(self, status):
        
        print(status)
        return True
    
try: 
    twitter_stream = Stream(auth, MyListener())
    twitter_stream.filter(track=['#bigdata'])
except KeyboardInterrupt:
    print("Streaming Feed Ended")
    